# 수치 미분의 문제점
 수치 미분을 이용하여 가중치 W, 바이어스 b를 업데이트 할 때의 문제점



1.   트레이닝 데이터 Training Data에 대하여
2.   피드 포워드 Feed Forward를 수행한 후에
3.   손실 함수를 계산하고, 손실 함수가 최소가 아니면
4.   수치 미분 Nemerical Derivative 을 통해서 가중치 W, 바이어스 b 를 업데이트 한 후에, 다시 피드 포워드를 반복 수행하는 코드 구현

  

*   MNIST 인식을 위해 784개의 입력층 노드, 30개의 은닉층 노드, 10개의 출력층 노드를 가지는 딥러닝 아키텍처로 학습을 시키편 평균 20시간 이상이 걸림(1CPU 환경)



*   이러한 **수치 미분의 단점을 극복**하고, 딥러닝의 성능을 획기적으로 개선한 알고리즘이 바로 **오차역전파 Back Propagation** 이다.









# 오차역전파 Back Propagation의 개념



1.   **가중치 W** / **바이어스 b** 등이 변할 때, **최종 오차 E**가 얼마나 변하는지 나타내는 **dE/dW** 또는 **dE/db** 같은 **편미분** 식을
2.   **체인 룰**을 이용하여 **dE/dA3** * **dA3/dA2** * **dA2/dW** 등 **국소 Local 미분**으로 분리한 다음
3.   이 국소 미분을 수학 공식으로 나타내서, 최종적으로 수치 미분이 아닌 **곱셈 형태의 산술식**으로 계산하는 방법

즉, **편미분 Partial Derivative 식**을 그대로 계산하는 것이 아니라, **체인 룰 Chain Rule**을 이용하여 국소 미분으로 분리한 다음, 이렇게 분리되어 있는 국소 미분을 계산하기 쉬운 형태의 수학공식으로 나타낸 것

특히, 딥러닝에서 오차역전파는 수치 미분을 사용하지 않고, **행렬 Matrix**로 된 수학 공식을 쓰기 때문에 빠른 계산이 가능하다



In [1]:
import numpy as np

def sigmoid(x) :
  return 1 / (1+np.exp(-x))

In [2]:
from datetime import datetime

class NeuralNetwork :

  def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate) :

    self.input_nodes = input_nodes
    self.hidden_nodes = hidden_nodes
    self.output_nodes = output_nodes

    self.W2 = np.random.randn(self.input_nodes, self.hidden_nodes) / np.sqrt(self.input_nodes/2)
    self.b2 = np.random.rand(self.hidden_nodes)

    self.W3 = np.random.randn(self.hidden_nodes, self.output_nodes) / np.sqrt(self.hidden_nodes/2)
    self.b3 = np.random.rand(self.output_nodes)

    self.Z3 = np.zeros([1, output_nodes])
    self.A3 = np.zeros([1, output_nodes])

    self.Z2 = np.zeros([1, hidden_nodes])
    self.A2 = np.zeros([1, hidden_nodes])

    self.Z1 = np.zeros([1, input_nodes])
    self.A1 = np.zeros([1, input_nodes])

    self.learning_rate = learning_rate


  def feed_forward(self) :

    delta = 1e-7

    self.Z1 = self.input_data
    self.A1 = self.input_data

    self.Z2 = np.dot(self.A1, self.W2) + self.b2
    self.A2 = sigmoid(self.Z2)

    self.Z3 = np.dot(self.A2, self.W3) + self.b3
    self.A3 = sigmoid(self.Z3)

    return -np.sum(self.target_data*np.log(self.A3+delta) + (1-self.target_data)*np.log((1-self.A3)+delta))


  def loss_val(self) :

    delta = 1e-7

    self.Z1 = self.input_data
    self.A1 = self.input_data

    self.Z2 = np.dot(self.A1, self.W2) + self.b2
    self.A2 = sigmoid(self.Z2)

    self.Z3 = np.dot(self.A2, self.W3) + self.b3
    self.A3 = sigmoid(self.Z3)

    return -np.sum(self.target_data*np.log(self.A3+delta) + (1-self.target_data)*np.log((1-self.A3)+delta))


  def accuracy(self, test_data) :

    matched_list = []
    not_matched_list = []

    for index in range(len(test_data)) :

      label = int(test_data[index, 0])

      data = (test_data[index, 1:] / 255.0 * 0.99) + 0.01

      predicted_num = self.predict(np.array(data, ndmin=2))

      if label == predicted_num :
        matched_list.append(index)
      else :
        not_matched_list.append(index)

    print("Currenc Accuracy = ", 100*(len(matched_list)/len(test_data)), "%")

    return matched_list, not_matched_list


# 출력층의 손실 = (출력층 츨력 - 정답) * 출력층 출력 *. (1 - 출력층 출력)
# 은닉층의 현재 손실 = (다음 층의 손실 * 다음 층 적용되는 가중치 W의 전치 T) * {현재층 출력 * (1 - 현재층 출력)}
# 현재층의 바이어스 변화율 = 현재층 손실
# 현재층에 적용되는 가중치의 변화율 = 이전층 출력의 전치 T * 현재층 손실


  def train(self, input_data, target_data) :

    self.target_data = target_data
    self.input_data = input_data

    loss_val = self.feed_forward()

    loss_3 = (self.A3 - self.target_data) * self.A3 * (1-self.A3)         # 출력층의 손실

    self.W3 = self.W3 - self.learning_rate * np.dot(self.A2.T, loss_3)    # 현재층에 적용되는 가중치의 변화율
    self.b3 = self.b3 - self.learning_rate * loss_3                       # 현재층의 바이어스 변화율

    loss_2 = np.dot(loss_3, self.W3.T) * self.A2 * (1-self.A2)            # 은닉층의 손실

    self.W2 = self.W2 - self.learning_rate * np.dot(self.A1.T, loss_2)    # 현재층에 적용되는 가중치의 변화율
    self.b2 = self.b2 = self.learning_rate * loss_2                       # 현재층의 바이어스 변화율


  def predict(self, input_data) :

    Z2 = np.dot(input_data, self.W2) + self.b2
    A2 = sigmoid(Z2)

    Z3 = np.dot(A2, self.W3) + self.b3
    A3 = sigmoid(Z3)

    predicted_num = np.argmax(A3)

    return predicted_num

In [3]:
training_data = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/AI/ML_DL_NeoWizard_Youtube/lecture_file/mnist_train.csv', delimiter=',', dtype=np.float32)
test_data = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/AI/ML_DL_NeoWizard_Youtube/lecture_file/mnist_test.csv', delimiter=',', dtype=np.float32)

In [4]:
print("training_data.shape = ", training_data.shape, ", test_data.shape = ", test_data.shape)

training_data.shape =  (60000, 785) , test_data.shape =  (10000, 785)


In [5]:
print("training_data[0,0] = ", training_data[0,0], ", test_data[0,0] = ", test_data[0,0])

print("len(training_data[0]) = ", len(training_data[0]), ", len(test_data[0]) = ", len(test_data[0]))

training_data[0,0] =  5.0 , test_data[0,0] =  7.0
len(training_data[0]) =  785 , len(test_data[0]) =  785


In [6]:
input_nodes = 784
hidden_nodes = 100
output_nodes = 10
learning_rate = 0.3
epochs = 1

nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

start_time = datetime.now()

for i in range(epochs) :

  for step in range(len(training_data)) :

    target_data = np.zeros(output_nodes) + 0.01
    target_data[int(training_data[step, 0])] = 0.99

    input_data = ((training_data[step, 1:] / 255.0) * 0.99) + 0.01

    nn.train(np.array(input_data, ndmin=2), np.array(target_data, ndmin=2))

    if step % 400 == 0 :
      print("step = ", step, ", loss_val = ", nn.loss_val())

end_time = datetime.now()

print("\nelapsed time = ", end_time - start_time)

step =  0 , loss_val =  2.0271592017037734
step =  400 , loss_val =  1.8326821405812572
step =  800 , loss_val =  0.8831161705099481
step =  1200 , loss_val =  0.7465793905790422
step =  1600 , loss_val =  1.3739704595002682
step =  2000 , loss_val =  1.0351950726322376
step =  2400 , loss_val =  0.7227223336045475
step =  2800 , loss_val =  0.8418414359689379
step =  3200 , loss_val =  0.8346089176853271
step =  3600 , loss_val =  0.7619119102886769
step =  4000 , loss_val =  0.9426640575159864
step =  4400 , loss_val =  0.7398594820241436
step =  4800 , loss_val =  0.8360709947516506
step =  5200 , loss_val =  0.7911232328662166
step =  5600 , loss_val =  0.9448542148660379
step =  6000 , loss_val =  0.8328173810018065
step =  6400 , loss_val =  0.9148964987085584
step =  6800 , loss_val =  0.9017717713104361
step =  7200 , loss_val =  0.83409006395676
step =  7600 , loss_val =  0.9124595346041522
step =  8000 , loss_val =  0.9451193957525974
step =  8400 , loss_val =  0.774341074333

In [7]:
nn.accuracy(test_data)

Currenc Accuracy =  93.92 %


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,